# #1 Cloning

In [ ]:
%cd /root
!git clone https://github.com/harry2141985/InfiniteTalk.git
%cd InfiniteTalk

# #2 Requirements

In [ ]:
!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121
!pip install -U xformers==0.0.28 --index-url https://download.pytorch.org/whl/cu121
!pip install misaki[en]
!pip install ninja
!pip install psutil
!pip install packaging
!pip install flash_attn==2.7.4.post1
!pip install -r requirements.txt
!pip install librosa
!pip install soundfile

# #3 Fixing Librosa

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

# Load example audio from librosa
y, sr = librosa.load(librosa.example('trumpet'))

# Display waveform
plt.figure(figsize=(10, 4))
librosa.display.waveshow(y, sr=sr)
plt.title("Waveform of Example Audio")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.show()

print("Librosa is working fine! Sample rate:", sr)

# #4 ffmpeg Install

In [ ]:
!apt-get -y update
!apt-get install -y libglib2.0-0 libgl1
!apt-get install -y ffmpeg

# #5 Download Models

In [ ]:
!huggingface-cli download Wan-AI/Wan2.1-I2V-14B-480P --local-dir ./weights/Wan2.1-I2V-14B-480P
!huggingface-cli download TencentGameMate/chinese-wav2vec2-base --local-dir ./weights/chinese-wav2vec2-base
!huggingface-cli download TencentGameMate/chinese-wav2vec2-base model.safetensors --revision refs/pr/1 --local-dir ./weights/chinese-wav2vec2-base
!huggingface-cli download MeiGen-AI/InfiniteTalk --local-dir ./weights/InfiniteTalk

# #6 Fixing model

In [ ]:
# Open multitalk.py for editing
multitalk_file = "/root/InfiniteTalk/wan/multitalk.py"

with open(multitalk_file, "r") as f:
    lines = f.readlines()

new_lines = []
for line in lines:
    if "from inspect import ArgSpec" in line:
        new_lines.append("from inspect import FullArgSpec as ArgSpec  # fixed for Python 3.12\n")
    else:
        new_lines.append(line)

with open(multitalk_file, "w") as f:
    f.writelines(new_lines)

print("Patched multitalk.py: replaced ArgSpec with FullArgSpec for Python 3.12.")

In [ ]:
# Path to the file
file_path = "/root/InfiniteTalk/src/audio_analysis/wav2vec2.py"

# Read the file
with open(file_path, "r") as f:
    lines = f.readlines()

# Comment out the line that sets output_attentions
with open(file_path, "w") as f:
    for line in lines:
        if "self.config.output_attentions = True" in line:
            f.write("# " + line)  # comment it out
        else:
            f.write(line)

print("Line 'self.config.output_attentions = True' has been commented out.")

# Main Code

In [ ]:
!python app.py \
    --ckpt_dir weights/Wan2.1-I2V-14B-480P \
    --wav2vec_dir 'weights/chinese-wav2vec2-base' \
    --infinitetalk_dir weights/InfiniteTalk/single/infinitetalk.safetensors \
    --num_persistent_param_in_dit 0 \
    --motion_frame 9